In [60]:
import os

import scattertext as st
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy

from tqdm.auto import tqdm

In [62]:
nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')

In [56]:
roe_opinion = BeautifulSoup(
    requests.get('https://www.law.cornell.edu/supremecourt/text/410/113').content, 
    "html.parser"
).find('div', attrs={'class': 'bodytext'}).text

# Note: this is from an OCR performed via OSX preview
dobbs_draft_opinion = open('draft_opinion.txt').read()

In [66]:
case_df = pd.DataFrame([
    {'Title': 'Roe','Text': roe_opinion},
    {'Title': 'Dobbs','Text': dobbs_draft_opinion},
]).assign(
    Parse = lambda df: df.Text.apply(nlp)
)

In [77]:
corpus = st.CorpusFromParsedDocuments(
    case_df,
    category_col = 'Title',
    parsed_col = 'Parse'
).build().remove_infrequent_words(
    4
)

In [78]:

html = st.produce_scattertext_explorer(
    corpus,
    category='Roe', 
    not_category_name='Dobbs',
    minimum_term_frequency=0, 
    pmi_threshold_coefficient=0,
    width_in_pixels=1000, 
    transform=st.Scalers.dense_rank
)

open('./roe_vs_dobbs.html', 'w').write(html)

1697337

In [79]:
!open roe_vs_dobbs.html

In [80]:
corpus.get_num_terms()

3510